In [1]:
from ipywidgets import FileUpload

# Create a file upload widget
upload_widget = FileUpload()

# Display the file upload widget
display(upload_widget)


FileUpload(value={}, description='Upload')

In [2]:
import numpy as np
import nltk
import string
import random
import csv
import string
import pandas as pd
import sklearn

In [24]:
with open('swahilicleanchat.csv', 'r',encoding='utf-8') as f:
    contents=f.read()


In [25]:
df = pd.read_csv('swahilicleanchat.csv', encoding='utf-8')
print(df.head())

                                             Message  \
0                                           SEPTEMBA   
1  Naitabaruku kazi hii kwa mke wangu Jemimah Kag...   
2                                                Pia   
3  kila nilipohitaji msaada wao. Naomba Mwenyezi ...   
4                     busara na maneno ya kutia moyo   

                                              Intent  
0                                               2020  
1                                      Pretty Makena  
2  ninatoa shukrani za dhati kwa wasimamizi wangu...  
3                                 ninamshukuru Prof.  
4  tangu mwanzo hadi mwisho wa utafiti huu. Wengi...  


In [11]:
print(df.columns)

Index(['Message', 'Intent'], dtype='object')


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data into a pandas DataFrame
data = pd.read_csv('swahilicleanchat.csv')

# Split data into features (X) and target (y)
X = data.drop('Message', axis=1)
y = data['Intent']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check for missing values in X_train
print(X_train.isnull().sum())


Intent    91
dtype: int64


In [14]:
# Replace missing values in the 'Message' column with an empty string
df['Message'].fillna(value='', inplace=True)


In [30]:
nltk.download('punkt') #using the punkt tokenizer
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
sentence_tokens = nltk.sent_tokenize(contents)
word_tokens = nltk.word_tokenize(contents)

In [31]:
sentence_tokens[:3]

['Message,Intent\nSEPTEMBA,2020\nNaitabaruku kazi hii kwa mke wangu Jemimah Kagwiria na wanangu Peace Kinya,Pretty Makena\nPia,ninatoa shukrani za dhati kwa wasimamizi wangu Prof. Ireri Mbaabu na Prof. Kitula King’ei\nkila nilipohitaji msaada wao.',
 'Naomba Mwenyezi Mungu awabariki!',
 'Aidha,ninamshukuru Prof.\nbusara na maneno ya kutia moyo,tangu mwanzo hadi mwisho wa utafiti huu.']

In [33]:
word_tokens[:5]

['Message', ',', 'Intent', 'SEPTEMBA,2020', 'Naitabaruku']

In [34]:
#normalize and lemmatize to base form
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
swahili_punct_dict = dict((ord(punct), None) for punct in string.punctuation )
#remove_punct_dict =dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(swahili_punct_dict)))

In [35]:
greet_inputs = ('vipi', 'jambo', 'hali gani', 'sema aje')
greet_responses =('Freshi', 'Jambo', 'Salama', 'Shwari Kabisa','Sijambo')
def greet(sentence):
  for word in sentence.split():
    if word.lower() in greet_inputs:
      return random.choice(greet_responses)



In [36]:
#feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
def response(user_response):
  robo1_response =''
  swahili_stop_words = ['na','ya','kwa', 'ni', ]
  TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english')
  tfidf = TfidfVec.fit_transform(sentence_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx=vals.argsort()[0][-2]
  flat= vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if (req_tfidf ==0):
    robo1_response = robo1_response + "Samahani. Siwezi Kuelewa"
    return robo1_response
  else:
    robo1_response = robo1_response + sentence_tokens[idx]
    return robo1_response

In [ ]:
flag= True
print('Hello! I am your Chatbot. Niulize maswali.')
sentence_tokens = []
word_tokens = []
final_words = []
while(flag==True):
  user_response = input()
  user_response = user_response.lower()
  if (user_response !='kwaheri'):
    if (user_response =='asante' or user_response == 'shukrani'):
      flag= False
      print('Bot:Karibu..')
    else:
      if (greet(user_response) != None):
        print('Bot ' + greet(user_response))
      else:
        sentence_tokens.append(user_response)
        word_tokens = word_tokens + nltk.word_tokenize(user_response)
        final_words = list(set(word_tokens))
        print('Bot: ', end='')
        print(response(user_response))
        sentence_tokens.remove(user_response)
else:
   flag=False
   print('Bot: Kwaheri!')

Hello! I am your Chatbot. Niulize maswali.


In [ ]:

from sklearn.metrics import classification_report

# Load your dataset
dataset = pd.read_csv('swahilicleanchat.csv', encoding='utf-8')
# Initialize variables to keep track of evaluation metrics
correct_predictions = 0
total_predictions = 0
true_positives = 0
false_positives = 0
false_negatives = 0
bot_responses = []

# Loop through the dataset
for index, data in dataset.iterrows():
    # Generate a response from your chatbot
    bot_response = response(data['Message'])
    bot_responses.append(bot_response)
    # Check if the response is correct
    if bot_response == data['Intent']:
        correct_predictions += 1
        true_positives += 1
    else:
        false_positives += 1
        false_negatives += 1
    total_predictions += 1

# Calculate evaluation metrics
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)
accuracy = correct_predictions / total_predictions

# Print evaluation metrics
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1_score)
print('Accuracy:', accuracy)
print(classification_report([data['answer'] for data in dataset], bot_responses)) #classification report
